# Standalone Quantum Chemistry Molecular Integrals

The following file is the time-intensive part of the [MolecularIntegrals.jl]()
package. This is called thousands of times in a typical calculation, and every
bit of speed is important here.

I'd be grateful for any help people could give me on speeding things up. Thus far,
I have not had any luck speeding this up further using macros like `@simd`
or `@turbo`. Often one of these macros speeds up the timing of the `vrr!` routine, but slows down the overall code for [reasons I don't understand](https://discourse.julialang.org/t/turbo-speeds-routine-slows-down-everything-else/62163).

I've read through and checked everything in the [Julia Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/) page and @ChrisRackauckas [7 Julia Gotchas and How to Handle Them](https://www.stochasticlifestyle.com/7-julia-gotchas-handle/) blog post.

I believe there are still performance gains to be had, because the head to head timing of my code against those C/C++ libraries shows that the Julia code is still 5-10 times slower.

The best algorithms in common use make use of recurrence relations to generate integrals for higher angular momentum basis functions in terms of the lower angular momentum integrals, which are ultimate worked on in terms of incomplete error functions. These methods involve a certain amount of irregular memory access.


## Utility functions

In [ ]:
@inline factorial2(n::Int64) = prod(n:-2:1); # double factorial !!
@inline dist2(dxyz) = dot(dxyz,dxyz);
@inline dist2(xyz1,xyz2) = dist2(xyz1-xyz2);

"gammainc - return the lower incomplete gamma function"
@inline gammainc(a,x) = gamma(a)*gamma_inc(a,x)[1];

In [ ]:
function boys_array_gamma(mmax,T,SMALL=1e-18)
    T = max(T,SMALL) # needs underflow protection because of inverse
    boys_array = zeros(Float64,mmax)
    ooT = 1/T
    denom = sqrt(ooT)
    boys_array[1] = 0.5*denom*gammainc(0.5,T) 
    for m in 2:mmax
        denom *= ooT
        # Could speed this up more by expressing gamma(m) in terms of gamma(m±1)
        boys_array[m] = 0.5*denom*gammainc(m-0.5,T) 
    end
    return boys_array
end;